In [ ]:
#Game of 15
import random

# Function to print the Tic Tac Toe board
def print_board(board):
    for row in board:
        print("|".join(map(str, row)))
        print("-----")

# Function to check if a player has won
def check_win(board, player_pieces):
    # Check rows, columns, and diagonals
    for i in range(3):
        # Check rows
        row_sum = sum(board[i][j] for j in range(3) if board[i][j] != 0)
        if row_sum == 15 and board[i].count(0) == 0:
            return True

        # Check columns
        col_sum = sum(board[j][i] for j in range(3) if board[j][i] != 0)
        if col_sum == 15 and [board[j][i] for j in range(3)].count(0) == 0:
            return True

    # Check diagonals
    main_diag_sum = sum(board[i][i] for i in range(3) if board[i][i] != 0)
    anti_diag_sum = sum(board[i][2 - i] for i in range(3) if board[i][2 - i] != 0)
    if (main_diag_sum == 15 and [board[i][i] for i in range(3)].count(0) == 0) or \
       (anti_diag_sum == 15 and [board[i][2 - i] for i in range(3)].count(0) == 0):
        return True

    return False

# Function to get available moves
def available_moves(board):
    moves = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                moves.append((i, j))
    return moves

# Function to perform AI's first move
def ai_first_move(board, ai_pieces, user_pieces):
    # Randomly select a position for AI's first move
    move = random.choice(available_moves(board))
    board[move[0]][move[1]] = ai_pieces.pop()

# Function to perform user move
def user_move(board, user_piece, user_negative_piece):
    while True:
        try:
            row = int(input("Enter the row (1-3): ")) - 1
            col = int(input("Enter the column (1-3): ")) - 1
            piece = int(input(f"Enter your move (piece {user_piece + user_negative_piece}): "))
            
            if piece in user_piece + user_negative_piece and piece in range(-9, 10):
                if board[row][col] == 0:
                    # Update the board with the user's move
                    if piece > 0:
                        board[row][col] = user_piece.pop(user_piece.index(piece))
                        if -piece in user_negative_piece:
                            user_negative_piece.remove(-piece)  # Remove the negative counterpart
                    else:
                        board[row][col] = user_negative_piece.pop(user_negative_piece.index(piece))
                        user_negative_piece.clear()  # Clear all other negative numbers
                        if -piece in user_piece:
                            user_piece.remove(-piece)  # Remove the positive counterpart
                    print("Your move:")
                    print_board(board)
                    break
                else:
                    print("This position is already taken. Try again.")
            else:
                print("Invalid move. Please enter a valid unused piece.")
        except ValueError:
            print("Invalid input. Please enter a number.")


def ai_move(board, ai_pieces, user_pieces, user_negative_piece):
    # Check if there's already a negative number on the board
    negative_on_board = any(cell < 0 for row in board for cell in row)

    # Check if AI can win in the next move
    for move in available_moves(board):
        for piece_val in ai_pieces:
            new_board = [row[:] for row in board]
            new_board[move[0]][move[1]] = piece_val
            if check_win(new_board, ai_pieces):
                board[move[0]][move[1]] = piece_val
                print("AI's move:")
                print_board(board)
                return

    # Check if AI needs to block user's winning move
    for move in available_moves(board):
        for piece_val in user_pieces:
            new_board = [row[:] for row in board]
            new_board[move[0]][move[1]] = piece_val
            if check_win(new_board, user_pieces):
                board[move[0]][move[1]] = ai_pieces.pop()  # Use a positive number
                print("AI's move:")
                print_board(board)
                return

    # If no winning move or blocking move, choose a winning move with a negative number
    if not negative_on_board:
        for move in available_moves(board):
            for piece_val in ai_pieces:
                new_board = [row[:] for row in board]
                new_board[move[0]][move[1]] = -piece_val  # Use a negative number
                if check_win(new_board, ai_pieces):
                    board[move[0]][move[1]] = -piece_val
                    print("AI's move:")
                    print_board(board)
                    return

    # If no winning move with negative number or negative number on the board, choose a random move
    move = random.choice(available_moves(board))
    board[move[0]][move[1]] = ai_pieces.pop()
    print("AI's move:")
    print_board(board)




# Function to check for a draw condition
def is_draw(board, user_pieces, ai_pieces):
    return all(cell != 0 for row in board for cell in row) and not check_win(board, user_pieces)

# Main function to run the game
def main():
    # Initialize the board
    board = [[0 for _ in range(3)] for _ in range(3)]

    print("Let's start the game!")
    print_board(board)

    # Ask players to choose their pieces (odd or even)
    while True:
        user_choice = input("Choose your piece (odd or even): ").lower()
        if user_choice in ['odd', 'even']:
            break
        else:
            print("Invalid choice. Please enter 'odd' or 'even'.")

    if user_choice == 'odd':
        user_pieces = [1, 3, 5, 7, 9]
        user_negative_piece = [-1, -3, -5, -7, -9]
        ai_pieces = [2, 4, 6, 8]
        ai_negative_pieces = [-2, -4, -6, -8]
    else:
        user_pieces = [2, 4, 6, 8]
        user_negative_piece = [-2, -4, -6, -8]
        ai_pieces = [1, 3, 5, 7, 9]
        ai_negative_pieces = [-1, -3, -5, -7, -9]

    # AI makes the first move if the user chose even
    if user_choice == 'even':
        ai_first_move(board, ai_pieces, user_pieces)

    print_board(board)

    # Game loop
    while True:
        # User's turn
        user_move(board, user_pieces, user_negative_piece)
        if check_win(board, user_pieces):
            print("Congratulations! You win!")
            break
        if is_draw(board, user_pieces, ai_pieces):
            print("It's a draw!")
            break

        # AI's turn
        ai_move(board, ai_pieces, user_pieces, user_negative_piece)
        if check_win(board, ai_pieces):
            print("AI wins! Better luck next time!")
            break
        if is_draw(board, user_pieces, ai_pieces):
            print("It's a draw!")
            break

if __name__ == "__main__":
    main()
